Você faz parte da equipe de Analytics de uma grande marca de vestuário com mais de 25 lojas espalhadas em Shoppings de todo o Brasil.

Toda semana você precisa enviar para a diretoria um ranking atualizado com as 25 lojas contendo 3 informações:

Faturamento de cada Loja;  
Quantidade de produtos vendidos de cada Loja;  
Ticket médio dos produto de cada Loja.  

Além disso, cada loja tem 1 gerente que precisa receber o resumo das informações da loja dele. Por isso, cada gerente deve receber no e-mail:  

Faturamento da sua Loja;  
Quantidade de produtos Vendidos da sua Loja;  
Ticket médio dos produto da sua Loja.  
Esse relatório é sempre enviado como um resumo de todos os dados disponíveis no ano.  

In [2]:
import pandas as pd

In [4]:
df = pd.read_excel('vendas.xlsx')
df

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
100995,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
100996,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108
